In [ ]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

In [ ]:
using Revise
using TORA

In [ ]:
using LinearAlgebra
# Try to load something faster than OpenBLAS
try using AppleAccelerate catch e; end
try using MKL catch e; end
BLAS.get_config()

In [ ]:
# using KNITRO
using MeshCat
using RigidBodyDynamics

In [ ]:
# Load the pre-compiled binaries of Coin HSL routines
HSL_jll_path = expanduser("~/HSL_jll.jl-2023.5.26")
Pkg.develop(path=HSL_jll_path)
import HSL_jll

In [ ]:
TORA.greet()

In [ ]:
vis = Visualizer()

setprop!(vis["/Cameras/default/rotated/<object>"], "fov", 40)

# IJuliaCell(vis)  # Show the viewer here (in the notebook)
open(vis)  # Show the viewer in a separate tab

In [ ]:
robot = TORA.create_robot_franka("panda_arm", vis)
# robot = TORA.create_robot_kinova_gen2("j2s6s200", vis)
# robot = TORA.create_robot_kinova_gen3("gen3_robotiq_2f_140", vis)
# robot = TORA.create_robot_kuka("iiwa14", vis)
# robot = TORA.create_robot_ur("ur10e", vis)

problem = TORA.Problem(robot, 2001, 1/1000)

# Constrain initial and final joint velocities to zero
TORA.fix_joint_velocities!(problem, robot, 1, zeros(robot.n_v))
TORA.fix_joint_velocities!(problem, robot, problem.num_knots, zeros(robot.n_v))

# # Constrain the position of the end-effector
# TORA.constrain_ee_position!(problem,   1, [ 1.0,  0.0,  0.5])
# TORA.constrain_ee_position!(problem, 101, [ 0.0,  1.0,  0.5])
# TORA.constrain_ee_position!(problem, 201, [-1.0,  0.0,  0.5])
# TORA.constrain_ee_position!(problem, 301, [ 0.0, -1.0,  0.5])

let
    CubicTimeScaling(Tf::Number, t::Number) = 3(t / Tf)^2 - 2(t / Tf)^3
    QuinticTimeScaling(Tf::Number, t::Number) = 10(t / Tf)^3 - 15(t / Tf)^4 + 6(t / Tf)^5

    for k = 1:2:problem.num_knots
        θ = CubicTimeScaling(problem.num_knots - 1, k - 1) * 2π
        # pos = [0.5, 0.2 * cos(θ), 0.8 + 0.2 * sin(θ)]  # UR10e
        pos = [0.4, 0.2 * cos(θ), 0.7 + 0.2 * sin(θ)]  # Franka Emika
        # pos = [0.5, 0.3 * sin(θ) + 0.1 * sin(8 * θ), 0.8 + 0.3 * cos(θ) + 0.1 * cos(8 * θ)]
        TORA.constrain_ee_position!(problem, k, pos)
    end
end

TORA.show_problem_info(problem)

In [ ]:
initial_q = zeros(robot.n_q)
# initial_q = [0, 0, 0, -π/2, 0, 0, 0]
initial_q = [0, -π/4, 0, -3π/4, 0, π/2, 0]  # Franka Emika
# initial_q = [0, -120, 120, -180, -90, 0] .|> deg2rad  # UR10e
# initial_q = x[1:7]

zero!(robot.state)
set_configuration!(robot.state, initial_q)
set_configuration!(robot.mvis, configuration(robot.state))

initial_qs = repeat(initial_q, 1, problem.num_knots)
initial_vs = zeros(robot.n_v, problem.num_knots)
initial_τs = zeros(robot.n_τ, problem.num_knots)

initial_guess = [initial_qs; initial_vs; initial_τs]

# Flatten matrix and truncate torques of last knot
initial_guess = vec(initial_guess)[1:end - robot.n_τ];

In [ ]:
use_inv_dyn = true
minimise_torques = false

user_options = Dict(
    # === Termination === #  https://coin-or.github.io/Ipopt/OPTIONS.html#OPT_Termination
    # "tol" => 10e-2,  # default: 10e-8
    # "max_cpu_time" => 4.0,  # default: 10e20
    # "constr_viol_tol" => 0.1,  # default: 0.0001
    # "acceptable_tol" => 0.1,  # default: 10e-6
    # "acceptable_constr_viol_tol" => 0.1,  # default: 0.01

    # === Output === #  https://coin-or.github.io/Ipopt/OPTIONS.html#OPT_Output
    # "print_level" => 0,  # [0, 12], default: 5
    
    # === NLP Scaling === #  https://coin-or.github.io/Ipopt/OPTIONS.html#OPT_NLP_Scaling
    # "nlp_scaling_method" => "none",  # none, user-scaling, gradient-based (default), equilibration-based

    # === Warm Start === #  https://coin-or.github.io/Ipopt/OPTIONS.html#OPT_Warm_Start
    # "warm_start_init_point" => "yes",
    # "warm_start_same_structure" => "yes",
    # "warm_start_entire_iterate" => "yes",

    # === Barrier Parameter Update === #  https://coin-or.github.io/Ipopt/OPTIONS.html#OPT_Barrier_Parameter_Update
    "mu_strategy" => "adaptive",  # monotone (default), adaptive
    # "mu_oracle" => "loqo",  # probing, loqo, quality-function (default)

    # === Linear Solver === #  https://coin-or.github.io/Ipopt/OPTIONS.html#OPT_Linear_Solver
    "linear_solver" => "ma57",  # ma27 (default), ma57, ma77, ma86, ma97, (...)
    # "ma57_pre_alloc" => 1.10,  # [1, Inf), 1.05 (default)

    # === Step Calculation === #  https://coin-or.github.io/Ipopt/OPTIONS.html#OPT_Step_Calculation
    # "mehrotra_algorithm" => "yes",  # yes, no (default)
    "fast_step_computation" => "yes",  # yes, no (default)

    # === Hessian Approximation === #  https://coin-or.github.io/Ipopt/OPTIONS.html#OPT_Hessian_Approximation
    "hessian_approximation" => "limited-memory",  # exact (default), limited-memory
    # "hessian_approximation_space" => "all-variables", # nonlinear-variables (default), all-variables

    # === Derivative Checker === #  https://coin-or.github.io/Ipopt/OPTIONS.html#OPT_Derivative_Checker
    # "derivative_test" => "first-order",  # none, first-order, second-order, only-second-order
)

# Choose which solver you want to use:
solve = TORA.solve_with_ipopt  # Uses Ipopt (https://github.com/coin-or/Ipopt)
# solve = TORA.solve_with_knitro  # Uses KNITRO (https://www.artelys.com/solvers/knitro/)

# Calling this will start the optimization.
cpu_time, x, solver_log = solve(problem, robot,
                                initial_guess=initial_guess,
                                use_inv_dyn=use_inv_dyn,
                                minimise_torques=minimise_torques,
                                user_options=user_options)

In [ ]:
TORA.play_trajectory(vis, problem, robot, x)

In [ ]:
TORA.plot_results(problem, robot, x)

In [ ]:
TORA.plot_log(solver_log)